<a href="https://colab.research.google.com/github/JawadSk12/ML-Lab/blob/main/ML%20LAB%20PROBLEM%20STATEMENT%204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1


In [4]:
test.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,750000,32,blue-collar,married,secondary,no,1397,yes,no,unknown,21,may,224,1,-1,0,unknown
1,750001,44,management,married,tertiary,no,23,yes,no,cellular,3,apr,586,2,-1,0,unknown
2,750002,36,self-employed,married,primary,no,46,yes,yes,cellular,13,may,111,2,-1,0,unknown
3,750003,58,blue-collar,married,secondary,no,-1380,yes,yes,unknown,29,may,125,1,-1,0,unknown
4,750004,28,technician,single,secondary,no,1950,yes,no,cellular,22,jul,181,1,-1,0,unknown


In [8]:
print(train.shape)
print(test.shape)

train.head(-10)


(750000, 18)
(250000, 17)


,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749985,749985,47,management,divorced,tertiary,no,0,no,no,unknown,9,jun,58,1,-1,0,unknown,0
749986,749986,27,blue-collar,married,secondary,no,-10,yes,no,unknown,15,may,180,2,-1,0,unknown,0
749987,749987,50,blue-collar,married,secondary,no,631,yes,no,unknown,14,may,114,1,-1,0,unknown,0
749988,749988,39,management,married,tertiary,no,4041,yes,no,cellular,4,jun,124,7,-1,0,unknown,0


In [7]:
train.columns


Index(['id', 'age', 'job', 'marital', 'education', 'default', 'balance',
       'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome', 'y'],
      dtype='object')

In [9]:
TARGET = "y"   # change if name is different
ID_COL = "id"       # change if name is different

X = train.drop([TARGET, ID_COL], axis=1)
y = train[TARGET]

X_test = test.drop([ID_COL], axis=1)


In [12]:
 from sklearn.preprocessing import LabelEncoder

In [13]:
cat_cols = X.select_dtypes(include="object").columns
print("Categorical columns:", list(cat_cols))

for col in cat_cols:
    le = LabelEncoder()
    combined = pd.concat([X[col], X_test[col]], axis=0).astype(str)
    le.fit(combined)

    X[col] = le.transform(X[col].astype(str))
    X_test[col] = le.transform(X_test[col].astype(str))


Categorical columns: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']


In [14]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [15]:
skf = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(X_test))


In [16]:
model_params = {
    "n_estimators": 1000,
    "learning_rate": 0.03,
    "max_depth": 6,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "random_state": 42,
    "tree_method": "hist"
}


In [21]:
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\nFold {fold + 1}")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = XGBClassifier(**model_params)

    model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
    )

    val_preds = model.predict_proba(X_val)[:, 1]
    oof_preds[val_idx] = val_preds

    auc = roc_auc_score(y_val, val_preds)
    print("AUC:", auc)

    test_preds += model.predict_proba(X_test)[:, 1] / skf.n_splits



Fold 1
AUC: 0.9675047151145321

Fold 2
AUC: 0.9662600521398435

Fold 3
AUC: 0.9663854209774508

Fold 4
AUC: 0.9674317850640272

Fold 5
AUC: 0.9667429232577439


In [22]:
overall_auc = roc_auc_score(y, oof_preds)
print("\nOverall CV ROC-AUC:", overall_auc)



Overall CV ROC-AUC: 0.9668636825227992


In [23]:
submission = pd.DataFrame({
    ID_COL: test[ID_COL],
    TARGET: test_preds
})

submission.to_csv("submission.csv", index=False)
submission.head()


,id,y
0,750000,0.002315
1,750001,0.109031
2,750002,0.000350
3,750003,0.000158
4,750004,0.030927
